# models

> This package will hold the GxExM learner for model training.

In [ ]:
#| default_exp GEMlearn

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler
from pathlib import Path

import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager
from warnings import warn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torcheval.metrics import MeanSquaredError,Mean, R2Score
from fastprogress import progress_bar,master_bar

from hybridpredictmaize22.GEMdataset import *

In [ ]:
import torch

class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, dropout=0.1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.dropout = dropout

        # Create a list of linear layers, with the correct input and output dimensions
        self.layers = torch.nn.ModuleList([torch.nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim) for i in range(num_layers - 1)])
        self.layers.append(torch.nn.Linear(hidden_dim, output_dim))

        # Initialize the weights and biases of the linear layers using the Xavier initialization method
        for layer in self.layers:
            torch.nn.init.xavier_uniform_(layer.weight)
            if layer.bias is not None:
                torch.nn.init.zeros_(layer.bias)

    def forward(self, x):
        # Apply the dropout layer to the input
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)

        # Iterate through the linear layers, applying each one to the input
        for layer in self.layers:
            x = layer(x)
            x = torch.nn.functional.relu(x)
            x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)

        return x

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()